In [ ]:
# Copyright 2023, Acadential, All rights reserved.

# 9-1. MNIST Dataset

## Import Torch, Torchvision Libraries

In [1]:
import torch 
import torchvision
import torchvision
import numpy as np 
from torchvision import transforms
from tqdm import tqdm 

Section 2-5에서 살펴보았듯이 Torchvision에서는 다양한 데이터셋들을 제공합니다. [torchvision dataset doc](https://pytorch.org/vision/main/datasets.html) 
1. Cifar10, Cifar100
2. MNIST
3. Fashion MNIST
4. Cityscapes

그 중에서 이번 실습에서는 MNIST 데이터셋을 사용해보도록 하겠습니다. [mnist dataset doc](https://pytorch.org/vision/main/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST) 

MNIST은 0\~9까지의 수를 분류하는 task에 사용되는 데이터셋입니다. 28x28의 크기를 가진 grayscale 이미지와 0~9까지의 label로 구성되어 있습니다.

## Load mnist dataset from torchvision

In [27]:
train_data = torchvision.datasets.MNIST(
    root='../.cache',
    train=True,
    download=True)

In [28]:
test_data = torchvision.datasets.MNIST(
    root='../.cache',
    train=False,
    download=True)

In [29]:
type(train_data)

torchvision.datasets.mnist.MNIST

In [30]:
type(test_data)

torchvision.datasets.mnist.MNIST

torchvision.datasets.mnist.MNIST은 어떤 클라스인지 살펴보겠습니다. \
(fn + F12 로 inspection 해보기)

Inspection해보면 MNIST는 torchvision.datasets.vision을 상속받은 클래스임을 알 수 있습니다. \
그럼 이 클라스는 어떤 클라스를 상속받았는지 살펴보겠습니다.

In [31]:
from torchvision.datasets.vision import VisionDataset

VisionDataset은 결국 torch.utils.data.Dataset을 상속받은 클래스라는 것을 확인할 수 있습니다.

# Data sample 예시

In [32]:
img, label = train_data[0]  # train_data의 첫번째 데이터

In [9]:
img  # png image

In [33]:
label  # label

5

In [34]:
img = np.array(img)  # numpy array로 변환

In [35]:
print(img.dtype)  # uint8 (0~255 integer)
print(img.shape)  # (height, width)

uint8
(28, 28)


MNIST은 흑백 이미지이기 때문에 height, width로만 구성됩니다.

In [36]:
# unit8 특징에 따라 이미지의 각 픽실값은0에서 255사이의 값을 가짐
print("Max = ", np.max(img))
print("Min = ", np.min(img))

Max =  255
Min =  0


## Transform

In [37]:
# Numpy array MNIST dataset을 PyTorch Tensor로 변환 그리고 0~1 사이 값으로 정규화.
to_tensor = transforms.ToTensor()


In [39]:
tensor = to_tensor(img)

In [15]:
print(type(tensor))
print(tensor.shape)
print("Max = ", torch.max(tensor)) 
print("Min = ", torch.min(tensor))

# input shape이 추가된 1x28x28이 됨.
# to_tensor를 적용하게되면 float타입으로 변환하게 되고 이 tensor의 값은 0에서 1사이의 값으로 표준화되는 것입니다.


<class 'torch.Tensor'>
torch.Size([1, 28, 28])
Max =  tensor(1.)
Min =  tensor(0.)


## Transform이 적용된 데이터셋

In [18]:
# instane 값을 만들때 Trasnsform 인자로 명시해서 적용이 가능하다.
train_data = torchvision.datasets.MNIST(
    root='../.cache',
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

test_data = torchvision.datasets.MNIST(
    root='../.cache',
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

In [17]:
img, label = train_data[0]
type(img)  # torch.Tensor

torch.Tensor

## DataLoader

In [19]:
# 데이터로더는 데이터셋을 입력 인자로 받고 미니배치의크기, number of worker, shuffling을 할지 등을 정의함

train_dataloader = torch.utils.data.DataLoader(
    train_data,  # dataset
    batch_size=64,
    num_workers=2,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    test_data,
    batch_size=64,
    num_workers=2,
    shuffle=False # we don't need to shuffle the test data
)

- batch_size: 한번에 학습할 데이터의 개수 (Mini-batch Stochastic Gradient Descent)
- num_workers: 데이터를 불러오고 전처리를 실행해주는 병렬 process의 개수.
- shuffle: Shuffle=True로 설정하면 Epoch마다 데이터가 섞입니다. Shuffle=False로 설정하면 Epoch마다 데이터가 섞이지 않습니다.

In [20]:
batch, label = next(iter(train_dataloader))
print("Batch shape = ", batch.shape)  # torch.Size([64=미니배치수, 1=인풋채널개수, 28=높이, 28=넓이])
print("Label shape = ", label.shape)  # torch.Size([64=미니배치수])
# 그래서 미니 배치, 인풋 채널 갯수, 하이트, 위스의 크기로 구성되어 있고
# 레이블은 미니 배치의 크기인 64인 벡터로 구성되어 있습니다

Batch shape =  torch.Size([64, 1, 28, 28])
Label shape =  torch.Size([64])


In [21]:
tbar = tqdm(train_dataloader)
# loop에서 매 iteration 마다 배치와 레이블이 샘플링 되며 이들을 모델 학습에
# 사용하게된다. 아래코드는 psudo 코드로써 .


for batch, label in tbar:
    # 1. input X인 배치가 모델의 입력 인자로 넣어줘서 forward pass되어 prediction 값을 구하게 되고 
    # pred = model(batch)
    
    # 2. prediction 값은 레이블 값과 비교되어 손실함수 값을 구하게되고 
    # loss = loss_fn(pred, label)
    
    # 3. 이렇게 구한 손실함수 값을 backward pass 해서 이 손실함수 값에 대한 경사인, loss gradiant를 계산해준다.
    # loss.backward()
    
    # 4. 이렇게 계산된 loss gradient를 기반으로 우리가 gradient decent를 수행할 수 있게된다.
    # optimizer.step()
    pass 

100%|██████████| 938/938 [00:06<00:00, 141.55it/s]
